# Prepare input CytoSpace

In [3]:
global_path = './cell-cell-communication/' ## Path to the github downloaded repository

In [1]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import diopy
out_path = global_path + 'Data/Brain/Input_cytoSPACE/'


## Convert to AnnData and Human

This data was downloaded from the example of CellTrek, so it is in Seurat format, we need to convert it. Moreover, we need to map the genes to human (we download the mapping from BioMart)

In [20]:
# adata_st = diopy.input.read_h5(file = global_path+'Data/Brain/stdata.h5', assay_name='spatial')
# adata_ref = diopy.input.read_h5(file = global_path+'Data/Brain/scdata.h5')


# mouse2hum = pd.read_csv(global_path + 'Data/mouse2human.csv')
# mouse2hum = mouse2hum.dropna(subset = ['Human gene name'])
# mouse2hum = mouse2hum.dropna(subset = ['Gene name'])
# m2h = dict(zip(mouse2hum['Gene name'].tolist(), mouse2hum['Human gene name'].tolist()))

# ## Mapping
# gene_h = []
# for i in adata_ref.var.index:
#     try:
#         gene_h.append(m2h[i])
#     except:
#         gene_h.append(i)

# adata_ref.var['genes.human'] = gene_h
# adata_ref.var = adata_ref.var.reset_index().set_index('genes.human')

# gene_h = []
# for i in adata_st.var.index:
#     try:
#         gene_h.append(m2h[i])
#     except:
#         gene_h.append(i)

# adata_st.var['genes.human'] = gene_h
# adata_st.var = adata_st.var.reset_index().set_index('genes.human')

# adata_ref.write(global_path + 'Data/Brain/scdata.h5ad')
# adata_st.write(global_path + 'Data/Brain/stdata.h5ad')



# Load data

In [25]:
adata = sc.read(global_path + 'Data/Brain/scdata.h5ad')
adata_st = sc.read(global_path + 'Data/Brain/stdata.h5ad')
adata.var_names_make_unique()
adata_st.var_names_make_unique()


/home/epareja/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/epareja/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# Save raw scRNA-seq counts as tsv file

In [28]:
counts = pd.DataFrame(adata.layers['counts'].toarray()).T
counts.columns = adata.obs.index
counts.index = adata.var.index
counts.index.name='GENES'
counts.to_csv(out_path + 'GEX.tsv', sep = '\t')

# Add Celltype information

In [29]:
cell_label = pd.DataFrame(adata.obs.cell_type)
cell_label.columns = ['CellType']
cell_label.index.name='Cell IDs'
cell_label.to_csv(out_path + 'cell_label.tsv', sep = '\t')

# Save raw spatial counts

In [30]:
counts_sp = pd.DataFrame(adata_st.layers['counts'].toarray()).T
counts_sp.columns = adata_st.obs.index
counts_sp.index = adata_st.var.index
counts_sp.index.name='V1'
counts_sp.to_csv(out_path + 'GEX_sp.tsv', sep = '\t')

# Add coords

In [31]:
coords = pd.DataFrame(adata_st.obsm['spatial'])
coords.columns = ['row', 'col']
coords.index  = adata_st.obs.index
coords.index.name='SpotID'
coords.to_csv(out_path + 'coords.tsv', sep = '\t')

# Run cytoSPACE using the terminal (bash)

In [ ]:
cytospace \
   -sp ./Data/Breast/Input_cytoSPACE/GEX.tsv \
   -ctp ./Data/Breast/Input_cytoSPACE/cell_label.tsv \
   -stp ./Data/Breast/Input_cytoSPACE/GEX_sp.tsv \
   -cp ./Data/Breast/Input_cytoSPACE/coords.tsv \
   -o ./Results/Brain/CytoSPACE/
